In [ ]:
#Import necessary modules
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

#Plot matplotlib plots directly within a notebook
%matplotlib inline

In [ ]:
titanic_data = pd.read_csv('titanic_train.csv')

In [ ]:
# Target & basic feature selection (avoid leakage columns like 'alive')
target = "survived"
features = ["pclass","sex","age","sibsp","parch","fare","embarked","alone"]

df = df[features + [target]].dropna(subset=[target])  # ensure target present
X = df[features]
y = df[target].astype(int)  # ensure binary 0/1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(exclude=["number"]).columns.tolist()

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols),
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree = Pipeline([
    ("prep", preprocess),
    ("clf", DecisionTreeClassifier(random_state=42))
])

param_grid = {
    "clf__max_depth": [3, 5, 7, 9, None],
    "clf__min_samples_split": [2, 5, 10, 20],
    "clf__min_samples_leaf": [1, 2, 5, 10],
}

grid = GridSearchCV(
    estimator=tree,
    param_grid=param_grid,
    scoring={"acc": "accuracy", "f1": "f1"},
    refit="f1",                  # pick best by F1 (more informative than accuracy here)
    cv=5,
    n_jobs=-1,
    verbose=0
)

grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
print("Best CV F1:", grid.best_score_)
best_tree = grid.best_estimator_

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline([
    ("prep", preprocess),
    ("clf", RandomForestClassifier(
        n_estimators=300, max_depth=None, random_state=42, n_jobs=-1
    ))
])

rf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def eval_model(name, model, X_tr, y_tr, X_te, y_te):
    yhat_tr = model.predict(X_tr)
    yhat_te = model.predict(X_te)
    return {
        "Model": name,
        "Acc (train)": accuracy_score(y_tr, yhat_tr),
        "F1  (train)": f1_score(y_tr, yhat_tr),
        "Acc (test)":  accuracy_score(y_te, yhat_te),
        "F1  (test)":  f1_score(y_te, yhat_te),
    }

rows = []
rows.append(eval_model("DecisionTree (best)", best_tree, X_train, y_train, X_test, y_test))
rows.append(eval_model("RandomForest (300)", rf, X_train, y_train, X_test, y_test))
metrics_df = pd.DataFrame(rows).set_index("Model")
metrics_df.round(3)

In [ ]:
# Get feature names after preprocessing
prep = best_tree.named_steps["prep"]
ohe = prep.named_transformers_["cat"].named_steps["ohe"]
num_names = num_cols
cat_names = ohe.get_feature_names_out(cat_cols).tolist()
all_feature_names = num_names + cat_names

# Extract importances from the final tree step
clf = best_tree.named_steps["clf"]
importances = pd.Series(clf.feature_importances_, index=all_feature_names)
imp_sorted = importances.sort_values(ascending=False).head(15)

plt.figure(figsize=(8,5))
imp_sorted.plot(kind="barh")
plt.gca().invert_yaxis()
plt.title("Decision Tree Feature Importances (top 15)")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()